In [1]:
from coral.dataprovider import DataProvider
from coral import toolx
from Utils import *

In [2]:
from coral.arango_service import ArangoService
from pyArango.connection import *

In [3]:
__arango_conn = Connection(arangoURL='http://127.0.0.1:8529',username='root', password='echusOverlook')
arango_service = ArangoService(__arango_conn, 'eO')

In [4]:
aql = 'FOR item in SDT_Inventory RETURN item'
db = __arango_conn['eO']
data = db.AQLQuery(aql)

inv = Inventory('Inventory', None, {})
Inventory.root = inv

item_values = {}
item_name = ''
assembly = ''
subassembly = ''
count = 0

for i in range(len(data)):
    item_values = {}
    item_name = ''
    assembly = ''
    subassembly = ''
    
    item_values['mass'] = data[i]['Mass']
    item_values['volume'] = data[i]['Volume']
    item_values['power'] = data[i]['Power']
    item_values['cooling'] = data[i]['Cooling']
    item_values['crewtime'] = data[i]['Crewtime']
    item_name = data[i]['Item']
    assembly = data[i]['Assembly']
    subassembly = data[i]['Subassembly']
    count = data[i]['Count']
    for i in range(count):
        inv.add(item_name, item_values, assembly, subassembly)

In [5]:
print(inv.__repr__())

{
    "Transit to Mars": {
        "mass": 5165.730100000001,
        "volume": 11.100070233,
        "power": 0.616,
        "cooling": 0.685,
        "crewtime": 1.5,
        "BioM": {
            "mass": 399.16,
            "volume": 0,
            "power": 0,
            "cooling": 0,
            "crewtime": 0,
            "Crop reactor/Plant growth chamber": {
                "mass": 399.16,
                "volume": 0,
                "power": 0,
                "cooling": 0,
                "crewtime": 0
            },
            "Crop storage": {
                "mass": 0,
                "volume": 0,
                "power": 0,
                "cooling": 0,
                "crewtime": 0
            }
        },
        "Food": {
            "mass": 4766.570100000001,
            "volume": 11.100070233,
            "power": 0.616,
            "cooling": 0.685,
            "crewtime": 1.5,
            "Food - 210 (initial transit) + 50 (surface) + 210 (return)": {
             

In [6]:
aql = 'FOR item in SDT_RMA RETURN item'
db = __arango_conn['eO']
data = db.AQLQuery(aql)

constants = {}
segment_constants = {}

for i in range(len(data)):
    constants = {}
    constants['Leq'] = data[i]['Leq']
    constants['Veq'] = data[i]['Veq']
    constants['Peq'] = data[i]['Peq']
    constants['Ceq'] = data[i]['Ceq']
    constants['Teq'] = data[i]['Teq']
    constants['Number of days'] = data[i]['Length']
    segment_constants[i] = constants
    
path = [('Earth', 'Earth_orbit'), ('Earth_orbit', 'Mars_orbit'), ('Mars_orbit', 'Mars')]
tr1 = Segment('Transit to Mars', path, segment_constants[1], 'tr1')
path = [('Mars', 'Mars_orbit'), ('Mars_orbit', 'Earth_orbit'), ('Earth_orbit', 'Earth')]
tr2 = Segment('Return Transit', path, segment_constants[1], 'tr2')
path = [('Mars', 'Mars')]
sf = Segment('Surface Operations', path, segment_constants[2], 'sf')

scenario1 = Mission(tr1, sf, tr2)

In [7]:
scenario1.setInventoryFromObject(inv)

In [8]:
scenario1.ESM()

105000.76628072344

In [9]:
scenario1.xESM()

99852.25048072345

In [10]:
print(repr(scenario1))

Transit to Mars ESM: 58159.33
Surface Operations ESM: 6194.14
Return Transit ESM: 40647.3

TOTAL ESM: 105000.77
███████████████████████████████████████ tr1
█████ sf
████████████████████████████ tr2

Transit to Mars xESM: 58159.33
Surface Operations xESM: 1045.63
Return Transit xESM: 40647.3

TOTAL xESM: 99852.25
███████████████████████████████████████ tr1
█ sf
████████████████████████████ tr2


